In [1]:
# 📦 Importation des bibliothèques nécessaires
import pandas as pd                 # Pour manipuler les données tabulaires (DataFrames)
import numpy as np                  # Pour les opérations mathématiques (non utilisé ici mais souvent utile)
from sklearn.preprocessing import LabelEncoder  # Pour convertir les colonnes catégorielles en nombres
from lightgbm import LGBMClassifier # Modèle de classification rapide et puissant

# 📥 Chargement des jeux de données d'entraînement et de test
train = pd.read_csv('/kaggle/input/churn-detection/train.csv')  # Données avec la colonne cible "Churn"
test = pd.read_csv('/kaggle/input/churn-detection/test.csv')    # Données sans la colonne cible

# 🔖 Sauvegarder les IDs pour les réutiliser dans la soumission finale
test_ids = test['id'].copy()

# 🧠 Conversion de la cible "Churn" de texte à binaire : 'Yes' → 1, 'No' → 0
train['Churn'] = train['Churn'].map({'Yes': 1, 'No': 0})

# 🔢 Conversion de la colonne TotalCharges en numérique (peut contenir du texte vide ou invalide)
train['TotalCharges'] = pd.to_numeric(train['TotalCharges'], errors='coerce').fillna(0)
test['TotalCharges'] = pd.to_numeric(test['TotalCharges'], errors='coerce').fillna(0)

# 🔍 Identification des colonnes de type "objet" (texte) à encoder
cat_cols = train.select_dtypes(include='object').columns.tolist()

# 🔁 Encodage des colonnes catégorielles en nombres avec LabelEncoder (même encodage pour train et test)
for col in cat_cols:
    le = LabelEncoder()
    combined = pd.concat([train[col], test[col]]).astype(str)  # Combinaison pour éviter des valeurs inconnues
    le.fit(combined)                                           # Apprentissage des étiquettes
    train[col] = le.transform(train[col].astype(str))          # Transformation sur le train
    test[col] = le.transform(test[col].astype(str))            # Transformation sur le test

# 🧮 Séparation des features (X) et de la cible (y) dans les données d'entraînement
X = train.drop(columns=["id", "Churn"])  # On enlève l'ID et la cible
y = train["Churn"]                       # La colonne cible
X_test = test.drop(columns=["id"])       # On enlève juste l'ID pour les prédictions

# ⚙️ Création et entraînement du modèle LightGBM
model = LGBMClassifier(
    random_state=42,       # Pour avoir les mêmes résultats à chaque exécution
    class_weight="balanced",  # Gère les classes déséquilibrées automatiquement
    n_estimators=100       # Nombre d’arbres (plus = meilleur mais plus lent)
)
model.fit(X, y)  # Entraînement du modèle

# 📊 Prédictions : on récupère les probabilités de la classe "1" (churn = Yes)
probs = model.predict_proba(X_test)[:, 1]  # Probabilité que le client parte (churn)

# 🟢🟥 Transformation en classe 1 (Yes) ou 0 (No) selon un seuil de 0.5
final_preds = (probs > 0.5).astype(int)

# 🔄 Re-conversion des résultats binaires en texte "Yes"/"No" pour respecter le format du concours
churn_labels = pd.Series(final_preds).map({1: "Yes", 0: "No"})

# 💾 Création du fichier de soumission
submission = pd.DataFrame({
    "id": test_ids.astype(int),   # Les identifiants des clients
    "Churn": churn_labels         # Les prédictions sous forme de texte
})

# 💽 Export du fichier final au format CSV sans index
submission.to_csv("submission.csv", index=False)

# 🧪 Vérifications pour s’assurer que tout est correct
print("🧪 test_ids sample:", test_ids[:5].values)         # Affiche les premiers IDs
print("🧪 final_preds sample:", final_preds[:5])         # Affiche les premières prédictions numériques
print("✅ Lengths match:", len(test_ids) == len(final_preds))  # Vérifie que les longueurs correspondent


[LightGBM] [Info] Number of positive: 1495, number of negative: 4139
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002414 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 626
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
🧪 test_ids sample: [0 1 2 3 4]
🧪 final_preds sample: [0 1 0 1 0]
✅ Lengths match: True


In [2]:
print("🧪 test_ids sample:", test_ids[:5].values)
print("🧪 final_preds sample:", final_preds[:5])
print("✅ Lengths match:", len(test_ids) == len(final_preds))



🧪 test_ids sample: [0 1 2 3 4]
🧪 final_preds sample: [0 1 0 1 0]
✅ Lengths match: True
